In [2]:
from CBAStats.Player import *
from CBAStats.Team import *
from CBAStats.Player import stats_output
import datetime
from Scraping import Scraper
import pandas as pd
from pathlib import Path

In [3]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-01-20 08:02:08.398035


In [4]:
schedule_output = f'StatsData/GamesSchedulePage_{datetime.date.today()}.csv'
game_stats_output = f'StatsData/All_Games_Stats_{datetime.date.today()}.csv'
teams_stats_output = f'StatsData/TeamsAdvancedStats_{datetime.date.today()}.csv'
players_stats_output = f'StatsData/PlayersAdvancedStats_{datetime.date.today()}.csv'

In [5]:
for output in [schedule_output,game_stats_output,teams_stats_output,players_stats_output]:
    path = Path(output)
    print(f'Checking {output}.')
    # check if folder is reachable
    assert (path.parent.exists()),f"Folder '{path.parent}' does not exist."
    # check if file already exists
    if path.exists():
        print(f'\tFile {path} already exists! It will be overwritten.')
    print('\tPass!')

Checking StatsData/GamesSchedulePage_2020-01-20.csv.
	Pass!
Checking StatsData/All_Games_Stats_2020-01-20.csv.
	Pass!
Checking StatsData/TeamsAdvancedStats_2020-01-20.csv.
	Pass!
Checking StatsData/PlayersAdvancedStats_2020-01-20.csv.
	Pass!


# 更新数据

In [23]:
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
import pandas as pd
import datetime
import numpy as np
import re
import time

def get_page_content(url, encoding='UTF-8', remove_linebreaks=True
                     , header={
            'User-Agent': r'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                          r'Chrome/41.0.2227.1 Safari/537.36'}):
    session = requests.Session()
    base_url = url
    response = session.get(base_url, headers=header)
    response.encoding = encoding
    page_content = BeautifulSoup(response.content, "html.parser")
    page_content_str = str(page_content)
    if remove_linebreaks:
        page_content_str = re.sub('\n', '', page_content_str)
    return page_content_str

def get_team_details(page_content_str, re_pattern):
    team_details = re.findall(re_pattern, page_content_str)[0]
    return team_details


def get_table_coach(team_details):
    coach = re.findall('主教练：(.*?)领队', team_details)[0].replace('\xa0', '')
    return coach


def get_table_lingdui(team_details):
    lingdui = re.findall('领队：(.*?)<', team_details)[0].replace('\xa0', '')
    return lingdui


def get_table_headers(team_details):
    team_table_str = re.findall('<table>(.*?)</table>', team_details)[0]
    team_table_header_str = re.findall('<thead>(.*?)</thead>', team_table_str)[0]
    team_table_header_lst = re.findall('<th>(.*?)</th>', team_table_header_str)
    return team_table_header_lst


def clean_cells(cell_data):
    if '<a' in cell_data:
        cell_data = re.findall('<a.*?>(.*?)</a>', cell_data)[0]
    if 'document.write' in cell_data:
        try:
            cell_data = re.findall('[(]"(\d*)"[)]', cell_data)[0]
        except IndexError:
            cell_data = re.findall('[(]"(\d*/\d)"[)]', cell_data)[0]
    cell_data = re.sub('\s', '', cell_data)
    cell_data = re.sub('是', '1', cell_data)
    cell_data = re.sub('否', '0', cell_data)
    return cell_data


def get_table_cells(team_details, headers_count):
    team_table_str = re.findall('<table>(.*?)</table>', team_details)[0]
    team_table_cells_str = re.findall('<tbody style="">(.*?)</tbody>', team_table_str)[0]
    team_table_cells_lst = re.findall('<td>(.*?)</td>', team_table_cells_str)
    team_table_cells_lst = list(map(clean_cells, team_table_cells_lst))
    team_table_cells_lst = np.reshape(team_table_cells_lst, (-1, headers_count)).tolist()
    return team_table_cells_lst

def get_hometeam_stats(page_content_str, re_pattern='<div class="part part01 blk">(.*?)<div class="part part02 blk">'):
    hometeam_details = get_team_details(page_content_str, re_pattern)

    hometeam_coach = get_table_coach(hometeam_details)
    hometeam_lingdui = get_table_lingdui(hometeam_details)

    hometeam_table_header_lst = get_table_headers(hometeam_details)
    hometeam_table_cells_lst = get_table_cells(hometeam_details, len(hometeam_table_header_lst))

    df_hometeam_details = pd.DataFrame(data=hometeam_table_cells_lst, columns=hometeam_table_header_lst)
    return df_hometeam_details


def get_awayteam_stats(page_content_str, re_pattern='<div class="part part02 blk">(.*?)<div class="part part03 blk">'):
    awayteam_details = get_team_details(page_content_str, re_pattern)

    awayteam_coach = get_table_coach(awayteam_details)
    awayteam_lingdui = get_table_lingdui(awayteam_details)

    awayteam_table_header_lst = get_table_headers(awayteam_details)
    awayteam_table_cells_lst = get_table_cells(awayteam_details, len(awayteam_table_header_lst))

    df_awayteam_details = pd.DataFrame(data=awayteam_table_cells_lst, columns=awayteam_table_header_lst)
    return df_awayteam_details


def split_made_attempt(df_orig):
    df = df_orig.copy()
    for col_name in list(filter(lambda x: '-' in x, df.columns.tolist())):
        orig_col = col_name
        col_made = re.findall('(.*)中-投', col_name)[0] + '中'
        col_attempt = re.findall('(.*)中-投', col_name)[0] + '投'
        df[[col_made, col_attempt]] = df[orig_col].str.split('-', expand=True)
        df[col_attempt] = df[col_attempt].apply(lambda x: re.sub('[(].*[)]', '', x))
        df.drop(columns=orig_col, inplace=True)
    return df

def scrape_game_details(url,game_id):
    base_url = url
    df_list = []
    game_UID = game_id
    game_hteam_name = '广东'
    game_ateam_name = '新疆'
    page_content_str = get_page_content(base_url, encoding='GB2312')

    # Get home team stats
    df_home = get_hometeam_stats(page_content_str=page_content_str)
    df_home['Game_ID'] = game_UID
    df_home['球队'] = game_hteam_name
    df_home['对手'] = game_ateam_name
    df_list.append(df_home)

    # Get away team stats
    df_away = get_awayteam_stats(page_content_str=page_content_str)
    df_away['Game_ID'] = game_UID
    df_away['球队'] = game_ateam_name
    df_away['对手'] = game_hteam_name  # use to get opponents stats
    df_list.append(df_away)
    print('抓取比赛 ', game_UID, ' ', game_hteam_name, ' ', game_ateam_name, ' ', datetime.datetime.now())

    # pause for a few seconds to avoid getting banned
    time.sleep(np.random.rand() * 5)
    games_stats = pd.concat(df_list, ignore_index=True)
    # remove team stats
    if 2 * len(games_stats['Game_ID'].unique()) == len(games_stats.loc[games_stats['号码'] == '--']):
        games_stats.drop(games_stats.loc[games_stats['号码'] == '--'].index, inplace=True)

    games_stats = split_made_attempt(games_stats)
    print(f'各场比赛详细数据保存在output_file')
#     games_stats.to_csv('test.csv', encoding='UTF-8', index=False)
    return games_stats

In [41]:
all_links = {
    '2014-11-23':'http://cba.sports.sina.com.cn/cba/schedule/show/12520/',
    '2015-01-07':'http://cba.sports.sina.com.cn/cba/schedule/show/12896/',
    '2015-12-16':'http://cba.sports.sina.com.cn/cba/schedule/show/14095/',
    '2016-01-20':'http://cba.sports.sina.com.cn/cba/schedule/show/14221/',
    '2016-11-18':'http://cba.sports.sina.com.cn/cba/schedule/show/14649/',
    '2017-01-31':'http://cba.sports.sina.com.cn/cba/schedule/show/14843/',
    '2017-03-31':'http://cba.sports.sina.com.cn/cba/schedule/show/14995/',
    '2017-04-02':'http://cba.sports.sina.com.cn/cba/schedule/show/14996/',
    '2017-04-05':'http://cba.sports.sina.com.cn/cba/schedule/show/14997/',
    '2017-04-07':'http://cba.sports.sina.com.cn/cba/schedule/show/14998/',
    '2019-11-19':'http://cba.sports.sina.com.cn/cba/schedule/show/18446/'
}

In [42]:
for x in all_links:
    print(x,all_links[x])

2014-11-23 http://cba.sports.sina.com.cn/cba/schedule/show/12520/
2015-01-07 http://cba.sports.sina.com.cn/cba/schedule/show/12896/
2015-12-16 http://cba.sports.sina.com.cn/cba/schedule/show/14095/
2016-01-20 http://cba.sports.sina.com.cn/cba/schedule/show/14221/
2016-11-18 http://cba.sports.sina.com.cn/cba/schedule/show/14649/
2017-01-31 http://cba.sports.sina.com.cn/cba/schedule/show/14843/
2017-03-31 http://cba.sports.sina.com.cn/cba/schedule/show/14995/
2017-04-02 http://cba.sports.sina.com.cn/cba/schedule/show/14996/
2017-04-05 http://cba.sports.sina.com.cn/cba/schedule/show/14997/
2017-04-07 http://cba.sports.sina.com.cn/cba/schedule/show/14998/
2019-11-19 http://cba.sports.sina.com.cn/cba/schedule/show/18446/


In [140]:
stats_list =[]
for link in all_links:
    print(link,all_links[link])
    df = scrape_game_details(all_links[link],link)
    time.sleep(3)
    stats_list.append(df)

2014-11-23 http://cba.sports.sina.com.cn/cba/schedule/show/12520/
抓取比赛  2014-11-23   广东   新疆   2020-01-20 11:47:02.294030
各场比赛详细数据保存在output_file
2015-01-07 http://cba.sports.sina.com.cn/cba/schedule/show/12896/
抓取比赛  2015-01-07   广东   新疆   2020-01-20 11:47:10.271774
各场比赛详细数据保存在output_file
2015-12-16 http://cba.sports.sina.com.cn/cba/schedule/show/14095/
抓取比赛  2015-12-16   广东   新疆   2020-01-20 11:47:19.223118
各场比赛详细数据保存在output_file
2016-01-20 http://cba.sports.sina.com.cn/cba/schedule/show/14221/
抓取比赛  2016-01-20   广东   新疆   2020-01-20 11:47:25.835503
各场比赛详细数据保存在output_file
2016-11-18 http://cba.sports.sina.com.cn/cba/schedule/show/14649/
抓取比赛  2016-11-18   广东   新疆   2020-01-20 11:47:31.372326
各场比赛详细数据保存在output_file
2017-01-31 http://cba.sports.sina.com.cn/cba/schedule/show/14843/
抓取比赛  2017-01-31   广东   新疆   2020-01-20 11:47:38.364878
各场比赛详细数据保存在output_file
2017-03-31 http://cba.sports.sina.com.cn/cba/schedule/show/14995/
抓取比赛  2017-03-31   广东   新疆   2020-01-20 11:47:46.740017
各场比赛详细数据

In [183]:
full_df = pd.concat([df for df in stats_list],ignore_index=True)

In [184]:
full_df.query("号码=='--'").index

Int64Index([ 10,  11,  24,  34,  35,  45,  46,  57,  58,  68,  69,  79,  80,
             90,  91, 102, 103, 115, 116, 129, 130, 143, 144, 157, 158, 171,
            172, 183, 184, 195, 196, 209, 210, 220, 221, 232, 233, 243, 244],
           dtype='int64')

In [185]:
full_df = full_df.drop(full_df.query("号码=='--'").index).copy()
full_df[['出场分','出场秒']]=full_df['出场时间'].str.split(':',expand=True).copy()
full_df['出场时间'] = full_df['出场分'].copy()
full_df[['出场时间', '首发', '进攻篮板', '防守篮板', '助攻', '犯规', '抢断', '失误', '盖帽',
       '扣篮', '被侵', '得分', '2分中', '2分投', '3分中',
       '3分投', '罚球中', '罚球投', '出场分', '出场秒']]=full_df[['出场时间', '首发', '进攻篮板', '防守篮板', '助攻', '犯规', '抢断', '失误', '盖帽',
       '扣篮', '被侵', '得分', '2分中', '2分投', '3分中',
       '3分投', '罚球中', '罚球投', '出场分', '出场秒']].astype('float').copy()
full_df

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投,出场分,出场秒
0,布莱切,0,41.0,1.0,0.0,9.0,2.0,2.0,6.0,3.0,...,广东,新疆,10.0,15.0,6.0,9.0,5.0,7.0,41.0,19.0
1,周琦,9,28.0,1.0,4.0,3.0,1.0,1.0,1.0,0.0,...,广东,新疆,9.0,11.0,0.0,0.0,8.0,9.0,28.0,7.0
2,可兰白克,13,31.0,1.0,1.0,3.0,2.0,6.0,2.0,3.0,...,广东,新疆,1.0,2.0,1.0,4.0,0.0,0.0,31.0,35.0
3,刘炜,8,26.0,1.0,1.0,2.0,6.0,1.0,1.0,1.0,...,广东,新疆,2.0,5.0,0.0,1.0,0.0,0.0,26.0,27.0
4,刘书楠,51,21.0,1.0,0.0,3.0,0.0,3.0,1.0,1.0,...,广东,新疆,1.0,2.0,0.0,1.0,2.0,2.0,21.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,刘羽楠,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,新疆,广东,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
275,热甫卡提江-吐送江,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,新疆,广东,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
276,齐麟,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,新疆,广东,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
277,斯托克斯,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,新疆,广东,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [186]:
yi = full_df.query("球员=='易建联'")
yi

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投,出场分,出场秒
13,易建联,9,33.0,1.0,0.0,5.0,1.0,2.0,1.0,4.0,...,新疆,广东,7.0,12.0,0.0,2.0,3.0,3.0,33.0,21.0
27,易建联,9,42.0,1.0,7.0,14.0,3.0,2.0,1.0,2.0,...,广东,新疆,10.0,14.0,0.0,0.0,6.0,8.0,42.0,10.0
59,易建联,9,39.0,1.0,1.0,9.0,3.0,1.0,1.0,4.0,...,新疆,广东,9.0,14.0,1.0,1.0,1.0,3.0,39.0,37.0
70,易建联,9,42.0,1.0,2.0,9.0,0.0,5.0,1.0,3.0,...,广东,新疆,13.0,26.0,0.0,2.0,0.0,0.0,42.0,40.0
93,易建联,9,36.0,1.0,2.0,11.0,0.0,0.0,0.0,2.0,...,广东,新疆,5.0,13.0,3.0,4.0,2.0,2.0,36.0,3.0
161,易建联,9,30.0,1.0,1.0,11.0,4.0,1.0,1.0,2.0,...,新疆,广东,7.0,10.0,0.0,0.0,0.0,0.0,30.0,2.0
186,易建联,9,32.0,1.0,2.0,10.0,0.0,3.0,1.0,2.0,...,新疆,广东,4.0,7.0,1.0,4.0,8.0,9.0,32.0,36.0
198,易建联,9,36.0,1.0,3.0,13.0,0.0,2.0,1.0,3.0,...,广东,新疆,3.0,8.0,3.0,6.0,0.0,1.0,36.0,13.0
224,易建联,9,32.0,1.0,2.0,4.0,1.0,5.0,1.0,1.0,...,广东,新疆,5.0,10.0,0.0,2.0,3.0,6.0,32.0,24.0
247,易建联,9,35.0,1.0,4.0,10.0,0.0,3.0,0.0,1.0,...,广东,新疆,3.0,8.0,1.0,3.0,14.0,17.0,35.0,NaN


In [188]:
zhou = full_df.query("球员=='周琦'")

zhou

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投,出场分,出场秒
1,周琦,9,28.0,1.0,4.0,3.0,1.0,1.0,1.0,0.0,...,广东,新疆,9.0,11.0,0.0,0.0,8.0,9.0,28.0,7.0
49,周琦,9,41.0,1.0,4.0,5.0,0.0,4.0,0.0,0.0,...,广东,新疆,4.0,5.0,0.0,0.0,4.0,4.0,41.0,19.0
82,周琦,9,45.0,1.0,5.0,3.0,0.0,3.0,2.0,3.0,...,新疆,广东,3.0,7.0,1.0,1.0,5.0,8.0,45.0,46.0
117,周琦,9,34.0,1.0,2.0,6.0,0.0,2.0,1.0,2.0,...,广东,新疆,5.0,6.0,0.0,0.0,12.0,14.0,34.0,12.0
147,周琦,9,37.0,1.0,5.0,5.0,0.0,2.0,0.0,3.0,...,广东,新疆,3.0,9.0,2.0,3.0,8.0,10.0,37.0,56.0
174,周琦,9,43.0,1.0,6.0,5.0,2.0,1.0,1.0,6.0,...,广东,新疆,6.0,11.0,0.0,2.0,2.0,2.0,43.0,10.0
213,周琦,9,40.0,1.0,0.0,7.0,3.0,4.0,0.0,2.0,...,新疆,广东,2.0,4.0,0.0,2.0,3.0,3.0,40.0,50.0
235,周琦,9,40.0,1.0,2.0,8.0,1.0,3.0,1.0,2.0,...,新疆,广东,4.0,7.0,1.0,2.0,7.0,10.0,40.0,27.0
264,周琦,9,41.0,1.0,2.0,5.0,2.0,4.0,2.0,1.0,...,新疆,广东,8.0,12.0,2.0,5.0,4.0,8.0,41.0,NaN


In [189]:
jiaofeng_zhou = zhou.loc[zhou['Game_ID'].isin(yi['Game_ID']),:]
jiaofeng_zhou

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投,出场分,出场秒
1,周琦,9,28.0,1.0,4.0,3.0,1.0,1.0,1.0,0.0,...,广东,新疆,9.0,11.0,0.0,0.0,8.0,9.0,28.0,7.0
49,周琦,9,41.0,1.0,4.0,5.0,0.0,4.0,0.0,0.0,...,广东,新疆,4.0,5.0,0.0,0.0,4.0,4.0,41.0,19.0
82,周琦,9,45.0,1.0,5.0,3.0,0.0,3.0,2.0,3.0,...,新疆,广东,3.0,7.0,1.0,1.0,5.0,8.0,45.0,46.0
147,周琦,9,37.0,1.0,5.0,5.0,0.0,2.0,0.0,3.0,...,广东,新疆,3.0,9.0,2.0,3.0,8.0,10.0,37.0,56.0
174,周琦,9,43.0,1.0,6.0,5.0,2.0,1.0,1.0,6.0,...,广东,新疆,6.0,11.0,0.0,2.0,2.0,2.0,43.0,10.0
213,周琦,9,40.0,1.0,0.0,7.0,3.0,4.0,0.0,2.0,...,新疆,广东,2.0,4.0,0.0,2.0,3.0,3.0,40.0,50.0
235,周琦,9,40.0,1.0,2.0,8.0,1.0,3.0,1.0,2.0,...,新疆,广东,4.0,7.0,1.0,2.0,7.0,10.0,40.0,27.0
264,周琦,9,41.0,1.0,2.0,5.0,2.0,4.0,2.0,1.0,...,新疆,广东,8.0,12.0,2.0,5.0,4.0,8.0,41.0,NaN


In [190]:
jiaofeng_zhou.Game_ID

1      2014-11-23
49     2015-12-16
82     2016-01-20
147    2017-03-31
174    2017-04-02
213    2017-04-05
235    2017-04-07
264    2019-11-19
Name: Game_ID, dtype: object

In [191]:
jiaofeng_yi = yi.loc[yi['Game_ID'].isin(zhou['Game_ID']),:]
jiaofeng_yi

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投,出场分,出场秒
13,易建联,9,33.0,1.0,0.0,5.0,1.0,2.0,1.0,4.0,...,新疆,广东,7.0,12.0,0.0,2.0,3.0,3.0,33.0,21.0
59,易建联,9,39.0,1.0,1.0,9.0,3.0,1.0,1.0,4.0,...,新疆,广东,9.0,14.0,1.0,1.0,1.0,3.0,39.0,37.0
70,易建联,9,42.0,1.0,2.0,9.0,0.0,5.0,1.0,3.0,...,广东,新疆,13.0,26.0,0.0,2.0,0.0,0.0,42.0,40.0
161,易建联,9,30.0,1.0,1.0,11.0,4.0,1.0,1.0,2.0,...,新疆,广东,7.0,10.0,0.0,0.0,0.0,0.0,30.0,2.0
186,易建联,9,32.0,1.0,2.0,10.0,0.0,3.0,1.0,2.0,...,新疆,广东,4.0,7.0,1.0,4.0,8.0,9.0,32.0,36.0
198,易建联,9,36.0,1.0,3.0,13.0,0.0,2.0,1.0,3.0,...,广东,新疆,3.0,8.0,3.0,6.0,0.0,1.0,36.0,13.0
224,易建联,9,32.0,1.0,2.0,4.0,1.0,5.0,1.0,1.0,...,广东,新疆,5.0,10.0,0.0,2.0,3.0,6.0,32.0,24.0
247,易建联,9,35.0,1.0,4.0,10.0,0.0,3.0,0.0,1.0,...,广东,新疆,3.0,8.0,1.0,3.0,14.0,17.0,35.0,NaN


In [192]:
jiaofeng = pd.concat([jiaofeng_yi,jiaofeng_zhou],ignore_index=True)
jiaofeng

jiaofeng=jiaofeng.drop(columns=['出场分','出场秒'])

jiaofeng_mean = jiaofeng.groupby('球员').mean()
jiaofeng_mean

jiaofeng_mean['2分命中率'] = jiaofeng_mean['2分中']/jiaofeng_mean['2分投']
jiaofeng_mean['3分命中率'] = jiaofeng_mean['3分中']/jiaofeng_mean['3分投']
jiaofeng_mean['总命中率'] = (jiaofeng_mean['2分中']+jiaofeng_mean['3分中'])/(jiaofeng_mean['2分投']+jiaofeng_mean['3分投'])
jiaofeng_mean['罚球命中率'] = jiaofeng_mean['罚球中']/jiaofeng_mean['罚球投']
jiaofeng_mean

,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,盖帽,扣篮,...,2分中,2分投,3分中,3分投,罚球中,罚球投,2分命中率,3分命中率,总命中率,罚球命中率
球员,,,,,,,,,,,,,,,,,,,,,
周琦,39.375,1.0,3.500,5.125,1.125,2.75,0.875,2.125,3.500,2.000,...,4.875,8.250,0.75,1.875,5.125,6.750,0.590909,0.4,0.555556,0.759259
易建联,34.875,1.0,1.875,8.875,1.125,2.75,0.875,2.500,1.125,2.375,...,6.375,11.875,0.75,2.500,3.625,4.875,0.536842,0.3,0.495652,0.743590


In [117]:
jiaofeng_mean.to_csv('易建联周琦交锋历史数据.csv')